# Designing a Python library for building prototypes around MinHash

This is very much work-in-progress. May be the software and or ideas presented with be the subject of a peer-reviewed or self-published write-up. For now the URL for this is: https://github.com/lgautier/mashing-pumpkins

MinHash in the context of biological sequenced was introduced by the Maryland Bioinformatics Lab [add reference here].

Building a MinHash is akin to taking a sample of all k-mers / n-grams found in a sequence and using that sample as a signature or sketch for that sequence.

## A look at convenience *vs* performance

Moving Python code to C leads to performance improvement... sometimes.

### Test sequence

First we need a test sequence. Generating a random one quickly can be achieved as follows, for example. If you already have you own way to generate a sequence, or your own benchmark sequence, the following code cell can be changed so as to end up with a variable `sequence` that is a `bytes` object containing it.

In [1]:
# we take a DNA sequence as an example, but this is arbitrary and not necessary.
alphabet = b'ATGC'
# create a lookup structure to go from byte to 4-mer
# (a arbitrary byte is a bitpacked 4-mer)
quad = [None,]*(len(alphabet)**4)
i = 0
for b1 in alphabet:
    for b2 in alphabet:
        for b3 in alphabet:
            for b4 in alphabet:
                quad[i] = bytes((b1,b2,b3,b4))
                i += 1
# random bytes for a 1M genome (order of magnitude for a bacterial genome)
import ssl
size = int(1E6)
sequencebitpacked = ssl.RAND_bytes(int(size/4))
sequence = bytearray(int(size))
for i, b in zip(range(0, len(sequence), 4), sequencebitpacked):
    sequence[i:(i+4)] = quad[b]
sequence = bytes(sequence)

### Kicking the tires with `sourmash`

The executable `sourmash` is a nice package from the dib-lab implemented in Python and including a library [add reference here]. Perfect for trying out quick what MinHash sketches can do.

We will create a MinHash of maximum size 1000 (1000 elements) and of k-mer size 21 (all ngrams of length 21 across the input sequences will be considered for inclusion in the MinHash. At the time of writing MinHash is implemented in C/C++ and use that as a reference for speed, as we measure the time it takes to process our 1M reference sequence

In [2]:
from sourmash_lib._minhash import MinHash

import timeit

stmt = """
smh = MinHash(1000, 21)
smh.add_sequence(sequence)
"""
t_sourmash = timeit.timeit(stmt,
                           setup='from __main__ import sequence, MinHash; sequence=sequence.decode("utf-8")',
                           number=5)
print("%.2f seconds" % t_sourmash)

1.53 seconds


This is awesome. The sketch for a bacteria-sized DNA sequence can be computed very quickly (less than a second on my laptop).

### Redisigning it all for convenience and flexibility

We have redesigned what a class could look like, and implemented that design in Python
foremost for our own convenience and to match the claim of convenience. Now how bad is the impact on performance ?

Our new design allows flexibility with respect to the hash function used, and to initially illustrate our point we use `mmh` an existing Python package wrapping MurmurHash3, the hashing function used in `MASH` and `sourmash`.

In [3]:
import mmh3
hashfun = mmh3.hash128

from mashingpumpkins.minhashsketch import MaxHashNgramSketch

stmt = 'mhs = MaxHashNgramSketch(21, 1000, hashfun); mhs.add(sequence)'
t_basic = timeit.timeit(stmt,
                        setup='from __main__ import MaxHashNgramSketch, hashfun, sequence',
                        number=5)
print("%.2f seconds" % t_basic)

print("Our Python implementation is %.2f times slower." % (t_basic / t_sourmash))

1.38 seconds
Our Python implementation is 0.90 times slower.


Ah. Our Python implementation only using `mmh3` and the standard library is not slower.
We will assume that we are faster then.

**note: ** the careful reader will not that we have a MaxHash rather than a MinHash. This is so to use algorithms in the Python stan

In [4]:
print("Our Python implementation is %.2f times faster." % (t_sourmash / t_basic))

Our Python implementation is 1.11 times faster.


There is more to it though. The code in "mashingpumpkins" is doing more by keeping track of the k-mer/n-gram along with the hash value in order to allow the generation of inter-operable sketch [add reference to discussion on GitHub].

We can modifying our class to stop storing the associated k-mer (only keep the hash value) to see if it improves performances:

In [5]:
class NoNgrams(MaxHashNgramSketch):
    
    def _make_elt(self, h, ngram):
        return (h, )

stmt = 'mhs = NoNgrams(21, 1000, hashfun); mhs.add(sequence)'
t_nongrams = timeit.timeit(stmt,
                           setup='from __main__ import NoNgrams, hashfun, sequence',
                           number=5)

print("%.2f seconds" % t_nongrams)

print("Our Python implementation is %.2f times faster." % (t_sourmash / t_nongrams))

1.37 seconds
Our Python implementation is 1.12 times faster.


No real difference here. Storing the k-mers / n-grams only has an impact on the added memory required to stored the 1,000
k-mers of length 21.

The flexibility of our design comes from code modularity, and nested function calls can often have worse performances than inlined code. We inlined our code to observe the effect:

In [6]:
from heapq import heappush, heapreplace

class NoNgramsAndInlined(MaxHashNgramSketch):
    """
    Just like MaxHashNgramSketch but the method `add()` has all calls to other methods inlined.
    """


    def add(self, s):
        hashfun = self._hashfun
        heap = self._heap
        maxsize = self._maxsize
        nsize = self._nsize
        heapset = self._heapset
        i = None
        lheap = len(heap)
        if lheap > 0:
            heaptop = heap[0]
        else:
            heaptop = None
        for i in range(0, len(s)-nsize):
            ngram = s[i:(i+nsize)]
            h = hashfun(ngram)
            if lheap < maxsize:
                elt = h
                if elt not in heapset:
                    heapset.add(elt)
                    heappush(heap, elt)
                    heaptop = heap[0]
                    lheap += 1
            elif h  > heaptop:
                elt = h
                if elt not in heapset:
                    heapset = heapset
                    heapset.add(elt)
                    out = heapreplace(heap, elt)
                    heapset.remove(out)
                    heaptop = heap[0]
        if i is not None:
             self._nvisited += (i+1)    

import timeit
stmt = 'mhs = NoNgramsAndInlined(21, 1000, hashfun); mhs.add(sequence)'
t_nongrams_inlined = timeit.timeit(stmt,
                                   setup='from __main__ import NoNgramsAndInlined, hashfun, sequence',
                                   number=5)

print("%.2f seconds" % t_nongrams_inlined)

print("Our Python implementation is %.2f times faster." % (t_sourmash / t_nongrams_inlined))


1.30 seconds
Our Python implementation is 1.18 times faster.


Very marginal improvement, if significant at all, but arguably not really worth the trouble and the loss of flexibility.

The inlining is letting us explore and other idea though. Computing batches of hash values each time C is reached for MurmurHash3. We have implemented the small C function require to call MurmurHash for several k-mers, and our inlined implementation is easy to modify to take advantage of that:

In [ ]:
import array

class NoNgramsAndInlinedAndBatch(MaxHashNgramSketch):
    """
    Just like MaxHashNgramSketch but the method `add()` has all calls to other methods inlined.
    """


    def add(self, s, w=100):
        hashfun = self._hashfun
        heap = self._heap
        maxsize = self._maxsize
        nsize = self._nsize
        heapset = self._heapset
        i = None
        lheap = len(heap)
        hashbuffer = array.array('Q', [0,]*w)
        if lheap > 0:
            heaptop = heap[0]
        else:
            heaptop = None
        
        for i in range(0, len(s)-nsize, w):
            subs = s[i:(i+w)]
            nsubs = hashfun(subs, nsize, hashbuffer)
            for j in range(w):
                h = hashbuffer[j]
                if lheap < maxsize:
                    elt = h
                    if elt not in heapset:
                        heapset.add(elt)
                        heappush(heap, elt)
                        heaptop = heap[0]
                        lheap += 1
                elif h  > heaptop:
                    elt = h
                    if elt not in heapset:
                        heapset = heapset
                        heapset.add(elt)
                        out = heapreplace(heap, elt)
                        heapset.remove(out)
                        heaptop = heap[0]
        if i is not None:
             self._nvisited += (i+1)    

from mashingpumpkins._murmurhash3 import hasharray
hashfun = hasharray
stmt = 'mhs = NoNgramsAndInlinedAndBatch(21, 1000, hashfun); mhs.add(sequence)'
t_batch = timeit.timeit(stmt,
                        setup='from __main__ import NoNgramsAndInlinedAndBatch, hashfun, sequence',
                        number=5)

print("%.2f seconds" % t_batch)

print("Our Python implementation is %.2f times faster." % (t_nongrams_inlined / t_batch))


0.42 seconds
Our Python implementation is 3.10 times faster.


Wow. At the time of writing this is approximatively 3 times faster than C-implemented `sourmash`.

The next iteration on the design will include this, and will hopefully significantly faster than the original while
continuing to provide flexibility for research and prototypes.

## To infinite and beyond

Now how much time should it take to compute signature for various references ?

First we check quickly that the time is roughly proportional to the size of the reference:

In [ ]:
# random genome between 3 and 10 times larger than before

for size in (int(3E6), int(5E6), int(10E6)):
    print('\n* size: {:,}'.format(size))
    sequencebitpacked = ssl.RAND_bytes(int(size/4))
    sequence5 = bytearray(int(size))
    for i, b in zip(range(0, len(sequence5), 4), sequencebitpacked):
        sequence5[i:(i+4)] = quad[b]
    sequence5 = bytes(sequence5)

    stmt = """
smh = MinHash(1000, 21)
smh.add_sequence(sequence)
    """
    t_sourmash_5 = timeit.timeit(stmt,
                                 setup='from __main__ import sequence5, MinHash; sequence=sequence5.decode("utf-8")',
                                 number=5)
    print("%.2f seconds" % t_sourmash_5)
    print("size ratio: %.2f - time ratio: %.2f" % (len(sequence5)/len(sequence), t_sourmash_5/t_sourmash))

    hashfun = hasharray
    stmt = 'mhs = NoNgramsAndInlinedAndBatch(21, 1000, hashfun); mhs.add(sequence)'
    t_batch_5 = timeit.timeit(stmt,
                              setup='from __main__ import NoNgramsAndInlinedAndBatch, hashfun,'
                                    'sequence5; sequence=sequence5',
                              number=5)
    print("%.2f seconds" % t_batch_5)
    print("size ratio: %.2f - time ratio: %.2f" % (len(sequence5)/len(sequence), t_batch_5/t_batch))
    print("Our Python implementation is %.2f times faster." % (t_sourmash_5 / t_batch_5))


* size: 3,000,000
4.47 seconds
size ratio: 3.00 - time ratio: 2.92
1.21 seconds
size ratio: 3.00 - time ratio: 2.87
Our Python implementation is 3.70 times faster.

* size: 5,000,000
7.42 seconds
size ratio: 5.00 - time ratio: 4.84
2.10 seconds
size ratio: 5.00 - time ratio: 5.00
Our Python implementation is 3.53 times faster.

* size: 10,000,000


The time spent appears proportional to the size of the input sequence within than range of sizes.

### Back-of-envelope time

This would allow us to compute the signatures for >1,800 bacterial genomes per hour and per CPU. On a laptop 
(where this notebook was initially run). Using the 2 cores, this would be >3,600 bacterial genomes per hour.

When considering raw reads from a sequencing run, this would put the 5E11 bases optimally out of a latest
Illumina MiSeq sequencer processed on 4 core in about 13 hours. 7 hours on 8 cores. Not great, but quite bearable
when an 8-core machine can be rented per hour on cloud for cheap... and this is still in Python so customization
and ideas can be implemented rarther easily and quickly.